<a href="https://colab.research.google.com/github/fatimafsalazard-sketch/Proyectos/blob/main/Comentarios/ComentariosYoutube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Clasificador de comentarios

###Con este codigo se clasifican los comentarios de un video en Youtube


In [ ]:
!pip install transformers
!pip install youtube-comment-downloader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 8.0 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline
from itertools import islice
from youtube_comment_downloader import *
downloader = YoutubeCommentDownloader()

Positive_classifier = pipeline("text-classification", model="UMUTeam/roberta-spanish-sentiment-analysis")
Toxicity_classifier = pipeline("text-classification", model="textdetox/xlmr-large-toxicity-classifier-v2")


comments = downloader.get_comments_from_url('https://www.youtube.com/watch?v=f0yQwQc9MyQ', sort_by=SORT_BY_POPULAR)

AcumPositive = 0
AcumNegative = 0
AcumNeutral  = 0

AcumToxicity = 0

NumComments = 100


#Ciclo FOR para ir viendo cada uno de los comentarios
for comment in islice(comments, NumComments):

    votes_str = comment['votes'].replace(',', '') # Remove commas first
    votes = 0 # Default value in case of unparseable string

    # Handle '萬' for ten thousands (e.g., '2.6萬' becomes 26000)
    if '萬' in votes_str:
        num_part = float(votes_str.replace('萬', ''))
        votes = int(num_part * 10000)
    else:
        votes = int(float(votes_str))

    Positive_result = Positive_classifier(comment['text'])
    Toxicity_result = Toxicity_classifier(comment['text'])

    if Positive_result[0]['label']   == 'neutral':
      AcumNeutral = AcumNeutral + votes + 1
    elif Positive_result[0]['label'] == 'positive':
      AcumPositive = AcumPositive + votes + 1
    elif Positive_result[0]['label'] == 'negative':
      AcumNegative = AcumNegative + votes + 1

    if Toxicity_result[0]['label'] == 'LABEL_1':  #LABEL_1 significa toxico
      AcumToxicity = AcumToxicity + votes + 1

# Avoid division by zero if no comments were processed
total_comments_votes = AcumNegative + AcumNeutral + AcumPositive

PorcentajeNeg = (AcumNegative / total_comments_votes) * 100
PorcentajePos = (AcumPositive / total_comments_votes) * 100
PorcentajeNeu = (AcumNeutral  / total_comments_votes) * 100

PorcentajeTox = (AcumToxicity / total_comments_votes) * 100

print("Porcentaje de comentarios positivos: ", PorcentajePos)
print("Porcentaje de comentarios negativos: ", PorcentajeNeg)
print("Porcentaje de comentarios neutros: ", PorcentajeNeu)

print("Porcentaje de comentarios toxicos: ", PorcentajeTox)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Device set to use cpu


Porcentaje de comentarios positivos:  5.657894736842105
Porcentaje de comentarios negativos:  49.22932330827068
Porcentaje de comentarios neutros:  45.11278195488722
Porcentaje de comentarios toxicos:  15.86466165413534


In [ ]:
from transformers import pipeline
from itertools import islice
from youtube_comment_downloader import *

# Configuracion
downloader = YoutubeCommentDownloader()

print("Cargando modelos... (esto puede tardar un poco)")
# Textos muy largos
Emotion_classifier = pipeline("text-classification", model="michellejieli/emotion_text_classifier", top_k=None)
Toxicity_classifier = pipeline("text-classification", model="textdetox/xlmr-large-toxicity-classifier-v2")

# URL del video
url = 'https://www.youtube.com/watch?v=kffacxfA7G4&list=RDkffacxfA7G4&start_radio=1'
comments = downloader.get_comments_from_url(url, sort_by=SORT_BY_POPULAR)

# Variables acumuladoras
emotions_acum = {
    'Alegria': 0, 'Miedo': 0, 'Desagrado': 0, 'Tristeza': 0,
    'Ira': 0, 'Sorpresa': 0, 'Neutral': 0
}
acum_toxicity = 0
total_weighted_votes = 0
NumComments = 500

# Funcion de error

def parse_votes(vote_str):
    """Convierte strings como '1.2K', '5M' o '3.4萬' a numeros enteros."""
    if not vote_str: return 0
    s = vote_str.replace(',', '') # Quitar comas
    multiplier = 1

    if 'K' in s or 'k' in s:
        multiplier = 1000
        s = s.replace('K', '').replace('k', '')
    elif 'M' in s or 'm' in s:
        multiplier = 1000000
        s = s.replace('M', '').replace('m', '')
    elif '萬' in s:
        multiplier = 10000
        s = s.replace('萬', '')

    try:
        return int(float(s) * multiplier)
    except ValueError:
        return 0

def get_label(prediction):
    # Si devuelve lista de listas
    if isinstance(prediction, list) and len(prediction) > 0:
        if isinstance(prediction[0], list):
            return prediction[0][0]['label']
        # Si devuelve lista simple
        return prediction[0]['label']
    # Si devuelve diccionario directo
    return prediction['label']

# Procesamiento de datos
print(f"Analizando los primeros {NumComments} comentarios...")

# enumerate para saber en que numero vamos
for i, comment in enumerate(islice(comments, NumComments)):

    text = comment['text']

    # Obtener votos
    votes = parse_votes(comment['votes'])

    weight = votes + 1

    # Analisis de emocion
    try:
        emotion_output = Emotion_classifier(text, truncation=True)
        top_emotion = get_label(emotion_output)

        # Sumar al acumulador correspondiente si la etiqueta existe
        if top_emotion in emotions_acum:
            emotions_acum[top_emotion] += weight
    except Exception as e:
        print(f"Error procesando emocion en comentario {i}: {e}")

    # Analisis de toxicidad
    try:
        toxicity_output = Toxicity_classifier(text, truncation=True)
        top_toxicity = get_label(toxicity_output)

        if top_toxicity == 'LABEL_1':
            acum_toxicity += weight
    except Exception as e:
        print(f"Error procesando toxicidad en comentario {i}: {e}")

    total_weighted_votes += weight

# Resultados
print("\n" + "="*40)
print("RESULTADOS DEL ANALISIS")
print("="*40)

if total_weighted_votes > 0:
    print(f"Total de participacion analizada (Votos + Comentarios): {total_weighted_votes}\n")

    # Imprimir porcentajes de emociones
    for emotion, score in emotions_acum.items():
        percentage = (score / total_weighted_votes) * 100
        print(f"Porcentaje {emotion.capitalize()}: {percentage:.2f}%")

    print("-" * 20)

    # Imprimir porcentaje de toxicidad
    perc_tox = (acum_toxicity / total_weighted_votes) * 100
    print(f"Porcentaje de comentarios TOXICOS: {perc_tox:.2f}%")
else:
    print("No se encontraron comentarios o hubo un error en la descarga.")


Cargando modelos... (esto puede tardar un poco)


Device set to use cpu
Device set to use cpu


Analizando los primeros 500 comentarios...

RESULTADOS DEL ANÁLISIS
Total de participación analizada (Votos + Comentarios): 70395

Porcentaje Alegria: 0.00%
Porcentaje Miedo: 0.00%
Porcentaje Desagrado: 0.00%
Porcentaje Tristeza: 0.00%
Porcentaje Ira: 0.00%
Porcentaje Sorpresa: 0.00%
Porcentaje Neutral: 0.00%
--------------------
Porcentaje de comentarios TÓXICOS: 2.31%
